# **<center>Vehicle Detection and tracking with Background Subtraction and Contours.<center>**

    


In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [52]:
# load a video
cap = cv2.VideoCapture('Videos/Dataset/v1.mp4')
backgroundobject = cv2.createBackgroundSubtractorMOG2 (history=20,varThreshold=20, detectShadows = True )

while(1):
    ret, frame = cap.read()  
    if not ret:
        break
        
        
    # apply the background object on each frame
    fgmask = backgroundobject.apply(frame)

    # also extracting the real detected foreground part of the image (optional)
    real_part = cv2.bitwise_and(frame,frame,mask=fgmask)
    
    # making fgmask 3 channeled so it can be stacked with others
    fgmask_3 = cv2.cvtColor(fgmask, cv2.COLOR_GRAY2BGR)
    
    # Stack all three frames and show the image
    stacked = np.hstack((fgmask_3,frame,real_part))
    cv2.imshow('All three',cv2.resize(stacked,None,fx=0.65,fy=0.65))
 
    k = cv2.waitKey(30) &  0xff
    if k == 27:
        break
   
cap.release()
cv2.destroyAllWindows()

In [50]:

video = cv2.VideoCapture('Videos/Dataset/traffic.avi')

kernel = None

backgroundObject = cv2.createBackgroundSubtractorMOG2(detectShadows = True)
#backgroundobject = cv2.createBackgroundSubtractorMOG2 (history=20,varThreshold=20, detectShadows = True )

while True:
    ret, frame = video.read()
    #print(frame).
    
    if not ret:
        

        break

    # Apply the background object on the frame to get the segmented mask. 
    fgmask = backgroundObject.apply(frame)
    #initialMask = fgmask.copy()
    
    _, fgmask = cv2.threshold(fgmask, 250, 255, cv2.THRESH_BINARY)
    #noisymask = fgmask.copy()
    
    # Apply some morphological operations 
    fgmask = cv2.erode(fgmask, kernel, iterations = 1)
    fgmask = cv2.dilate(fgmask, kernel, iterations = 2)
    
    # Detect contours in the frame.
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    

    frameCopy = frame.copy()
    

    for cnt in contours:
        
        if cv2.contourArea(cnt) > 400:
            
            x, y, width, height = cv2.boundingRect(cnt)
            
            # Draw a bounding box around the car.
            cv2.rectangle(frameCopy, (x , y), (x + width, y + height),(0, 0, 255), 2)

            cv2.putText(frameCopy, 'Car Detected', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,255,0), 1, cv2.LINE_AA)
   
    foregroundPart = cv2.bitwise_and(frame, frame, mask=fgmask)
    
    stacked = np.hstack((frame, foregroundPart, frameCopy))


    cv2.imshow('Original Frame, Extracted Foreground and Detected Cars', cv2.resize(stacked, None, fx=0.5, fy=0.5))
    #cv2.imshow('initial Mask', initialMask)
    #cv2.imshow('Noisy Mask', noisymask)
    #cv2.imshow('Clean Mask', fgmask)


    k = cv2.waitKey(1) & 0xff
    
    if k == ord('q'):

        break


video.release()

cv2.destroyAllWindows()